# Ćwiczenia 4 - Regularyzacja

## Early stopping
Polega na zatrzymaniu uczenia w momencie kiedy wybrana metryka przestała się poprawiać dla danych walidacyjnych.
Keras pozwala na zaimplementowac tę metodę jako callback, który można podać jako parametr w metodzie `fit`, np.:

```python
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=2, restore_best_weights=True, verbose=1)

model.fit(..., callbacks=[early_stopping])
```

## Regularyzacja L1/L2

Do funkcji błędu dodawany jest dodatkowy termin penalizujący zbyt duże wagi:

$$ \tilde{J}(W) = J(W) + \lambda \sum_i w_i^2 $$
$$ \tilde{J}(W) = J(W) + \lambda \sum_i |w_i| $$

Aby zastosować tę metodę dla warstwy, należy ustawić parametr `kernel_regularizer`, np:

```python
tf.keras.layers.Dense(..., kernel_regularizer=tf.keras.regularizers.l2(0.1))
```

## Regularyzacja Dropout

Regularyzacja Dropout polega na losowym wyłączaniu pewnej części neuronów w sieci podczas uczenia.
Metodę tę można zaimplementowac dodając warstwę `Dropout` bezpośrednio po regularyzowanej warstwie.

## Augmentacja danych

Polega na wprowadzaniu drobnych zmian w danych treningowych.
Technika ta jest szczególnie przydatna w przetwarzaniu obrazów. Różne transformacje można zastosować dodając odpowiednie warstwy na wejściu sieci, np.: `RandomRotation`, `RandomZoom`, `RandomBrightness`. Pełna lista dostępncyh transformacji: https://keras.io/api/layers/preprocessing_layers/image_augmentation/

## Zadania
1. Wczytaj zbiór `imdb` z Ćwiczeń 03. Zastosuj sieć wielowarstwową do klasyfikacji zbioru (możesz wykorzystać sieć z przykładu). Stwórz wykres pokazujący błąd dla danych treningowych/walidacyjnych w kolejnych epokach i zaobserwuj efekt przetrenowania. Następnie uruchom sieć ponownie, stosując Early Stopping aby zatrzymać uczenie w momencie, kiedy funkcja straty przestanie się zminiejszać dla danych walidacyjnych.
2. Dodaj do warstw ukrytych sieci z Zadania 1. regularyzację L1/L2. Dobierz odpowiednie wartości współczynnika regularyzacji. Trenuj sieci bez stosowania Early Stopping. Stwórz wykresy krzywych uczenia i porównaj działanie regularyzowanych sieci z siecią z Zadania 1.
3. Dodaj do warstw ukrytych sieci z Zadania 1. regularyzację Dropout. Dobierz odpowiedni parametr `rate`. Stwórz wykres krzywej uczenia i porównaj działanie sieci stosującej Dropout z siecią z Zadania 1.
4. (opcjonalne) Dodaj do sieci klasyfikującej zbiór CIFAR-10 stworzonej na wcześniejszych zajęciach augmentację obrazów. Bezpośrednio po warstwie `Input` i przed warstwą `Flatten` dodaj wybrane transformacje.

In [13]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

import matplotlib.pyplot as plt

In [10]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

mlb = MultiLabelBinarizer()
mlb.fit(train_data + test_data)
X_train_full = mlb.transform(train_data)
y_train_full = train_labels
X_test = mlb.transform(test_data)
y_test = test_labels

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, test_size=0.33, random_state=911)

In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.05),
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [12]:
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=32, epochs=20)

Epoch 1/20
524/524 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.7096 - loss: 0.5498 - val_accuracy: 0.7784 - val_loss: 0.4976
Epoch 2/20
524/524 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.8803 - loss: 0.2950 - val_accuracy: 0.8588 - val_loss: 0.3384
Epoch 3/20
524/524 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9064 - loss: 0.2351 - val_accuracy: 0.8250 - val_loss: 0.4413
Epoch 4/20
524/524 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.9258 - loss: 0.1896 - val_accuracy: 0.8739 - val_loss: 0.2988
Epoch 5/20
524/524 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9421 - loss: 0.1540 - val_accuracy: 0.8755 - val_loss: 0.3159
Epoch 6/20
524/524 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.9484 - loss: 0.1369 - val_accuracy: 0.8773 - val_loss: 0.3195
Epoch 7/20
524/524 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.9598 - loss: 0.1091 - val_accuracy: 0.8725 - val_loss: 0.3667
Epoch 8/20
524/524 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9657 - loss: 0.0963 - val_accuracy: 

In [ ]:
fig, [ax1, ax2] = plt.subplots(1, 2, figsize=(10, 5))

loss = history.history["loss"]
val_loss = history.history["val_loss"]
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
print(history.history.keys())
epoch = range(len(loss))

ax1.plot(epoch, loss, label="Training Loss")
ax1.plot(epoch, val_loss, label="Validation Loss")
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Loss")
ax1.legend()

ax2.plot(epoch, accuracy, label="Training Accuracy")
ax2.plot(epoch, val_accuracy, label="Validation Accuracy")
ax2.set_xlabel("Epoch")
ax2.set_ylabel("Loss")
ax2.legend()